In [64]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA  
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
import pinecone
import os
from langchain.vectorstores import Pinecone
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from langchain.utilities import GoogleSerperAPIWrapper


In [65]:
api_key = os.environ.get("OPENAI_API_KEY")
serper_api = os.environ.get("SERPER_API_KEY")



# initialize pinecone
pinecone_api = os.environ.get("PINECONE_API_KEY")

pinecone.init(
    api_key=pinecone_api, 
    environment="gcp-starter"  
)

In [66]:
players_file = 'data/merged.csv'
loader = CSVLoader(file_path=players_file)
players = loader.load()

In [67]:
# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size = 300,
#     chunk_overlap  = 50,
#     length_function = len,
#     is_separator_regex = False,
# )

# texts = text_splitter.split_documents(players)

In [68]:
embeddings = OpenAIEmbeddings()

In [69]:
index_name = "players"
docsearch = Pinecone.from_existing_index(index_name, embeddings)


In [70]:
# query = "Who is the best QB?"
# docs = docsearch.similarity_search(query)
# print(len(docs))
# print(docs[0])
# print(docs[0].page_content)


In [71]:
team_template = """You are an expert in Sports Analytics and Data Science. The column "rank_ecr" indicates the Expert Consenus ranking, use this to determine how good the player is

You must adhere to the following rules:
1. There can be one QB, two RBs, three WRs, one TE, one DST and one FLEX who can be either a RB, WR or TE. 
2. You cannot exceed the total salary.
3. Provide the total salary spent in your answer

Use the following pieces of context to answer the question at the end. 
{context}
Question: {question}
Answer:"""
TEAM_PROMPT = PromptTemplate.from_template(team_template)

In [72]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)

In [73]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": TEAM_PROMPT},
    )

In [74]:
#defining the tools for the agent
tools = [
        Tool(
        name = "query",
        func=qa.run,
        description="use this as the primary source of context information when you are asked the question. Always search for the answers using this tool first, don't make up answers yourself"
        )
]

#Setting up the agent 
agent_chain = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [75]:
agent_chain.run(input="What is the best lineup of players for less than $50000?")



> Entering new AgentExecutor chain...
I need to find information on the players and their prices to determine the best lineup.
Action: query
Action Input: "best lineup of players under $50000"
Observation: To determine the best lineup of players under $50000, we need to consider the players' rankings and salaries. Based on the given information, we have the following players:

1. kevinaustinjr. (WR) - rank_ecr: 270, salary: No Salary
2. johnnymundt (TE) - rank_ecr: 84, salary: 2500.0
3. j.d.mckissic (RB) - rank_ecr: 108, salary: No Salary
4. johnkellyjr. (RB) - rank_ecr: 163, salary: No Salary

To adhere to the rules, we can select the following lineup:

QB: Not specified
RB1: j.d.mckissic (RB) - rank_ecr: 108, salary: No Salary
RB2: johnkellyjr. (RB) - rank_ecr: 163, salary: No Salary
WR1: kevinaustinjr. (WR) - rank_ecr: 270, salary: No Salary
WR2: Not specified
WR3: Not specified
TE: johnnymundt (TE) - rank_ecr: 84, salary: 2500.0
DST: Not specified
FLEX: Not specified

The total s

'The best lineup of players for less than $50000 is as follows:\n\nQB: Not specified\nRB1: j.d.mckissic (RB) - rank_ecr: 108, salary: No Salary\nRB2: johnkellyjr. (RB) - rank_ecr: 163, salary: No Salary\nWR1: kevinaustinjr. (WR) - rank_ecr: 270, salary: No Salary\nWR2: Not specified\nWR3: Not specified\nTE: johnnymundt (TE) - rank_ecr: 84, salary: 2500.0\nDST: Not specified\nFLEX: Not specified\n\nThe total salary spent in this lineup is $2500.0.'